# APIM ❤️ MCP

## MCP Protected Resources Metadata - PRM

![flow](../../images/mcp-client-authorization.gif)

Playground to experiment the [Model Context Protocol](https://modelcontextprotocol.io/) with the [client authorization flow](https://modelcontextprotocol.io/specification/2025-03-26/basic/authorization#2-10-third-party-authorization-flow). 

In this flow, Azure API Management act both as an OAuth client connecting to the [Microsoft Entra ID](https://learn.microsoft.com/en-us/entra/architecture/auth-oauth2) authorization server and as an OAuth authorization server for the MCP client ([MCP inspector](https://modelcontextprotocol.io/docs/tools/inspector) in this lab).

Full documentation of the assets created in this lab and how they interact is documented in [Arch decisions summary](ARCH_SUMMARY.md) and in [MCP PRM Server implementation](src/mcp-prm-server/README.md)


⚠️ This lab implements the [MCP Authorization proposal](https://modelcontextprotocol.io/specification/2025-03-26/basic/authorization) and [RFC9729](https://datatracker.ietf.org/doc/html/rfc9728), this is as close to production-grade security for MCP servers as it gets, and is inspired by the amazing work by [blackchoey/remote-mcp-apim-oauth-prm](https://github.com/blackchoey/remote-mcp-apim-oauth-prm) - originally written in C#

### Prerequisites

- [Python 3.12 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Python environment](https://code.visualstudio.com/docs/python/environments#_creating-environments) with the [requirements.txt](../../requirements.txt) or run `pip install -r requirements.txt` in your terminal
- [An Azure Subscription](https://azure.microsoft.com/free/) with [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#contributor) + [RBAC Administrator](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#role-based-access-control-administrator) or [Owner](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#owner) roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and [Signed into your Azure subscription](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

▶️ Click `Run All` to execute all steps sequentially, or execute them `Step by Step`...


<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [ ]:
import os, sys, json, base64, random, string
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "uksouth"

aiservices_config = []

models_config = []

apim_sku = 'Basicv2'
apim_subscriptions_config = []

inference_api_path = "inference"  # path to the inference API in the APIM service
inference_api_type = "AzureOpenAI"  # options: AzureOpenAI, AzureAI, OpenAI, PassThrough
inference_api_version = "2025-03-01-preview"
foundry_project_name = deployment_name

# Generate a unique name for the app registration
app_registration_name = f"lab-{deployment_name}-{''.join(random.choices(string.ascii_letters + string.digits, k=8)).lower()}-app"

build = 0
prm_mcp_server_image = "mcp-prm-server"
prm_mcp_server_src = "../../shared/mcp-servers/prm-graphapi"

# In this lab we will generate AES keys for encryption and decryption of tokens.
# This is an experimental feature and should NOT be used in production!
encryption_iv = base64.b64encode(os.urandom(16)).decode('utf-8')
encryption_key = base64.b64encode(os.urandom(16)).decode('utf-8')
oauth_scopes = 'openid https://graph.microsoft.com/.default'

utils.print_ok('Notebook initialized')

<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [ ]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

<a id='3'></a>
### 3️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [ ]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "mcpEntraAppName": { "value": app_registration_name },
        "apimSku": { "value": apim_sku },
        "aiServicesConfig": { "value": aiservices_config },
        #"mcpPrmPath": { "value": mcpPrmPath }, ## Add a path you would like your MCP server to exposed as ##
        "modelsConfig": { "value": models_config },
        "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
        "inferenceAPIPath": { "value": inference_api_path },
        "inferenceAPIType": { "value": inference_api_type },
        "foundryProjectName": { "value": foundry_project_name },
        "oauthScopes": { "value": oauth_scopes },
        "encryptionIV": { "value": encryption_iv },
        "encryptionKey": { "value": encryption_key },
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

<a id='4'></a>
### 4️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [ ]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    # Monitoring
    log_analytics_id = utils.get_deployment_output(output, 'logAnalyticsWorkspaceId', 'Log Analytics Id')
    app_insights_name = utils.get_deployment_output(output, 'applicationInsightsName', 'Application Insights Name')
    app_insights_app_id = utils.get_deployment_output(output, 'applicationInsightsAppId', 'Application Insights App Id')
    
    # API Management
    apim_resource_name = utils.get_deployment_output(output, 'apimResourceName', 'APIM Resource Name')
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    
    # Container Infrastructure
    container_registry_name = utils.get_deployment_output(output, 'containerRegistryName', 'Container Registry Name')
    container_registry_login_server = utils.get_deployment_output(output, 'containerRegistryLoginServer', 'Container Registry Login Server')
    mcp_server_containerapp_name = utils.get_deployment_output(output, 'mcpServerContainerAppName', 'MCP Server Container App Name')
    mcp_server_containerapp_fqdn = utils.get_deployment_output(output, 'mcpServerContainerAppFQDN', 'MCP Server Container App FQDN')
    #mcp_server_url = utils.get_deployment_output(output, 'mcpServerURL', 'MCP Server URL')
    
    # Managed Identity
    managed_identity_name = utils.get_deployment_output(output, 'managedIdentityName', 'Managed Identity Name')
    managed_identity_client_id = utils.get_deployment_output(output, 'managedIdentityClientId', 'Managed Identity Client Id')
    
    # MCP Configuration
    entra_app_client_id = utils.get_deployment_output(output, 'mcpAppId', 'MCP App Client Id')
    entra_tenant_id = utils.get_deployment_output(output, 'mcpAppTenantId', 'Entra Tenant Id')
    mcp_api_endpoint = utils.get_deployment_output(output, 'mcpApiEndpoint', 'MCP API Endpoint')
    mcp_prm_endpoint = utils.get_deployment_output(output, 'mcpPrmEndpoint', 'MCP PRM Endpoint')
    
    # Set legacy variable names for compatibility with existing code
    weather_containerapp_resource_name = mcp_server_containerapp_name


<a id='5'></a>
### 5️⃣ Build and deploy the MCP Servers



In [ ]:
build = build + 1 # increment the build number

utils.print_info(f"To Inspect the build logs, run: az acr task logs --registry {container_registry_name}")
utils.run(f"az acr build --image {prm_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {prm_mcp_server_src}/Dockerfile {prm_mcp_server_src}/. --no-logs", 
          "PRM MCP Server image was successfully built", "Failed to build the PRM MCP Server image")
utils.run(f'az containerapp update -n {weather_containerapp_resource_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{prm_mcp_server_image}:v0.{build}" --set-env-vars AZURE_CLIENT_ID="{entra_app_client_id}" AZURE_TENANT_ID="{entra_tenant_id}" AZURE_MANAGED_IDENTITY_CLIENT_ID="{managed_identity_client_id}"', 
          "PRM MCP Server deployment succeeded", "PRM MCP Server deployment failed")


<a id='unauthorizedtest'></a>
### 🧪 Test the authorization **WITHOUT** a valid token

In [ ]:
# Unauthenticated call should fail with 401 Unauthorized
import requests

mcp_server_url = f"{apim_resource_gateway_url}/mcp"
utils.print_info("Calling sse endpoint WITHOUT authorization...")
utils.print_message(f"MCP Server Url : {mcp_server_url}")
response = requests.post(mcp_server_url, headers={"Content-Type": "application/json"})

if response.status_code == 401:
    utils.print_ok("Received 401 Unauthorized as expected")
    utils.print_ok(f"Response Headers: {response.headers}")
elif response.status_code == 200:
    utils.print_error("Call succeeded. Double check that validate-jwt policy has been deployed to sse endpoint")
else:
    utils.print_error(f"Unexpected status code: {response.status_code}")



<a id='vscode'></a>
### 🧪 Use [VS Code Github Copilot](https://github.com/features/copilot) to test the Authorization flow

### Execute the following steps
1. In VS Code, Press ***Ctrl + Shift + P*** to choose [MCP: Add Server] add the apim endpoint for MCP as HTTP, and give it a name
2. Shortly it will ask you automatically to sign in to Microsoft
3. After you're signed in with your Entra Account, the MCP server will reveal that it has 1 tool
4. Open Github Copilot, make sure you're in ***Agent*** mode, and ask a question such as ***who am i?***
5. If all goes well, you should be able to Allow it to use MCP tool to execute the command
6. The first time you run the lab, you'll be asked to provide consent and given a URL to open in a browser
7. Once successful on the browser, you should be able to ask Github Copilot to ***retry execution*** or ***try again now***
8. Voila!!


<a id='inspector'></a>
### 🧪 Use the [MCP Inspector](https://modelcontextprotocol.io/docs/tools/inspector) to test the Authorization flow

#### Execute the following steps:
1. Execute `npx @modelcontextprotocol/inspector` in a terminal
2. Access the provided URL in a browser (it should open automatically).
3. Set the transport type as `Streamable HTTP`
4. Provide the MCP server URL
5. Click in the `Open Auth Settings` button
6. Click on `Quick OAuth Flow`
7. You’ll see a sign-in screen or an “Application Access Request” screen asking for your consent to use your signed-in account. After reviewing the request, click "Allow" to proceed.
8. After being redirected back to the MCP Inspector, scroll down to the `Authentication Complete` step. Expand the `Access Tokens` section and copy the `access_token` value.
9. Expand the Authentication section on the left and paste the `access_token` into the Bearer Token parameter.
10. Click on "Connect" and verify that the Weather Tool is functioning properly.

<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.